In [81]:
import pandas as pd
import re
import numpy as np
from numpy.linalg import norm
from nltk.util import ngrams

In [82]:
# 后缀标着1的，是用moka-ai/m3e-base
# 后缀标着2的，是用thenlper/gte-large-zh
# 后缀标着3的，是用amu/tao-8k


query_df = pd.read_pickle('query_df_final.pkl') # 导入query数据，这里只有四季酒店的数据，1000条
df = pd.read_pickle('faq_df_final.pkl') # 导入FAQ数据，这个是逸燊做的，大概从1000条里面调了两百多条典型的出来当FAQ

original = pd.read_excel('~/Desktop/group1_final/完整版评论分类1.xlsx', sheet_name = '深圳四季')
scheme_df = pd.read_excel('~/Desktop/group1_final/scheme.xlsx', sheet_name = 'scheme')

In [83]:
query_df.head(10)

,评论序号,评论内容,评论内容_new,no,query_embedding
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,酒店挺不错的,1.0,"[[0.38339633, 0.6643063, 0.70754176, -0.102897..."
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,房间干净卫生,1.0,"[[-0.18294388, 0.45822287, 0.27038223, -0.9868..."
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,设备齐全,1.0,"[[0.26044473, 0.67715794, 0.4333286, -0.249953..."
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,床品舒适,1.0,"[[0.2529874, 0.5675511, -0.035674542, 0.109394..."
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,早餐精致好吃,1.0,"[[0.21657318, 0.9036295, 0.22838418, -0.464586..."
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,,1.0,"[[0.26622725, 0.3910116, 0.6029822, -0.2258512..."
1,2,前台接待的M美女很好,前台接待的M美女很好,2.0,"[[0.5532722, 1.0749547, 0.8143255, 0.12496547,..."
2,3,位置好，房间现代、干净、漂亮！,位置好,3.0,"[[-0.08265738, 0.83002406, 0.746209, 0.1638858..."
2,3,位置好，房间现代、干净、漂亮！,房间现代、干净、漂亮,3.0,"[[0.31723502, 0.5127885, 0.4110963, -0.5490853..."
2,3,位置好，房间现代、干净、漂亮！,,3.0,"[[0.26622725, 0.3910116, 0.6029822, -0.2258512..."


In [84]:
df.head(10)

,分类结果,分类关键句,faq_new,faq_embedding
0,房间-布局-不合理,一开始的房间风水怪怪的,房间-布局-不合理:一开始的房间风水怪怪的,"[[0.41805333, 0.42092174, 0.66800374, -0.09395..."
1,房间-布局-不合理,房间格局奇怪的不得了，一根大柱子在房间,房间-布局-不合理:房间格局奇怪的不得了，一根大柱子在房间,"[[0.3116825, 0.6138913, 0.69666356, -0.0623900..."
2,房间-布局-不合理,洗手间倒是挺大,房间-布局-不合理:洗手间倒是挺大,"[[0.24111979, 0.7401576, 0.7612051, -0.0776110..."
3,房间-布局-合理,老五星 品质还是在的；房间格局很好,房间-布局-合理:老五星 品质还是在的；房间格局很好,"[[0.8829135, 0.605288, 0.96933293, -0.27354595..."
4,房间-大小-合适,房间够大,房间-大小-合适:房间够大,"[[-0.08642605, 0.25674045, 0.71628565, -0.5995..."
5,房间-大小-合适,房间够大,房间-大小-合适:房间够大,"[[-0.08642605, 0.25674045, 0.71628565, -0.5995..."
6,房间-大小-偏大,厕所很大,房间-大小-偏大:厕所很大,"[[-0.13091731, 1.1027104, 0.75805694, -0.40695..."
7,房间-大小-偏大,房大，床垫舒服,房间-大小-偏大:房大，床垫舒服,"[[-0.005964607, 0.85491186, 0.59670156, -0.290..."
8,房间-大小-偏大,酒店房间宽敞,房间-大小-偏大:酒店房间宽敞,"[[0.26855567, 0.89291847, 1.0140823, -0.371595..."
9,房间-大小-偏大,深圳四季酒店在会展中心对面，周边有丽思卡尔顿和柏悦酒店，位置很赞。房间很大，位置很棒。但早餐一般,房间-大小-偏大:深圳四季酒店在会展中心对面，周边有丽思卡尔顿和柏悦酒店，位置很赞。房间很大...,"[[0.048785795, 0.8155682, 0.79713273, 0.183587..."


In [85]:
original.head(10)

,评论序号,评论内容,分类结果,分类关键句,分类关键词
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,"房间-清洁程度-干净,房间-舒适度-舒适,房间-设施-好",房间干净卫生；床品舒适；设备齐全,干净卫生；舒适；齐全
1,2,前台接待的M美女很好,服务-态度-好,M美女很好,很好
2,3,位置好，房间现代、干净、漂亮！,"位置-便利性-交通-发达,房间-清洁程度-干净",位置好；房间干净,好；干净
3,4,"前台的态度很好，升级了行政房 房间设备非常人性化，卫生间是自动感应的,房间温度调试的很舒适，...","服务-态度-好,房间-设施-好,位置-便利性-交通-发达,位置-便利性-附近设施多样性-多",前台的态度很好；房间设备非常人性化；酒店位置很好，周围有很多知名餐厅,"很好；人性化；很好,多"
4,5,目前全国住过印象最好的酒店,服务-态度-好,印象最好的酒店,最好
5,6,非常好的一次入住体验 前台办理入住的同事Susan非常高效 酒店的健身房和泳池也很棒 会再来...,"服务-专业性-高,酒店额外设施-质量-好,酒店额外设施-多样性-丰富",前台办理入住的同事Susan非常高效；酒店的健身房和泳池也很棒,高效；很棒
6,7,本来入住很满意，前台一个男性服务员服务态度差的要死。前台留错我的电话，一直和我争杠。,服务-态度-坏,前台一个男性服务员服务态度差的要死,服务态度差的要死
7,8,服务一如既往的好，老客户客房做了升级，这个也是一直选择四季的原因之一，早餐如果增加一些地方特...,"服务-态度-好,服务-专业性-高,价格-附加值服务-多",服务一如既往的好；客房做了升级；请注重一下门童形象、举止,很好；升级；形象、举止
8,9,位于福田中心的优秀酒店，靠近Coco Park。总的来说一切都很出色。可能会说英语的人多一点？,"位置-便利性-交通-发达,位置-便利性-附近设施多样性-多",位于福田中心的优秀酒店；靠近Coco Park,福田中心；靠近Coco Park
9,10,整体来说非常棒，服务非常好，各个方面都很不错，特别干净舒适，果然是5星酒店啊。早餐挺不错的，...,"房间-清洁程度-干净,房间-舒适度-舒适,服务-态度-好,位置-便利性-附近设施多样性-多,...",特别干净舒适；服务非常好；周边环境，商场都很多,"干净舒适；非常好；周边环境优美,商场很多"


In [86]:
scheme_df.head(10)

,level1,level2,level3,level4
0,房间,大小,偏大,NaN
1,NaN,NaN,合适,NaN
2,NaN,NaN,偏小,NaN
3,NaN,布局,合理,NaN
4,NaN,NaN,不合理,NaN
5,NaN,清洁程度,干净,NaN
6,NaN,NaN,不干净,NaN
7,NaN,舒适度,舒适,NaN
8,NaN,NaN,不适,NaN
9,NaN,隔音水平,安静,NaN


In [87]:
# 定义一个余弦相似度，用于计算两个语句之间的相似度
def cosine_similarty(a,b):
    res = np.dot(a,b)/(norm(a)*norm(b))
    return res 

In [88]:
def split_(row, col):
    if pd.isnull(row[col]) == False:
        res = re.split('！|，|。|, |？|,', row[col])
    else:
        res = None
    return res 

In [89]:
def scheme_level(row):
    if pd.isnull(row['level2']) == True:
        res = row['level1']
    elif pd.isnull(row['level3']) == True:
        res = row['level1']+'-'+row['level2']
    elif pd.isnull(row['level4']) == True:
        res = row['level1']+'-'+row['level2'] +'-'+row['level3']      
    else:
        res = row['level1']+'-'+row['level2'] +'-'+row['level3']  +'-'+row['level4']          
    return res

In [90]:
def range_list(start, end, step):
    res = []
    current_value = start

    while current_value <= end:
        res.append(current_value)
        current_value  = current_value + step
        
    return res

In [91]:
original = original.rename(columns = {'评论序号':'no'})
original['分类结果_new'] = original.apply(split_, axis=1, col = '分类结果')
original = original.explode('分类结果_new')
original


,no,评论内容,分类结果,分类关键句,分类关键词,分类结果_new
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,"房间-清洁程度-干净,房间-舒适度-舒适,房间-设施-好",房间干净卫生；床品舒适；设备齐全,干净卫生；舒适；齐全,房间-清洁程度-干净
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,"房间-清洁程度-干净,房间-舒适度-舒适,房间-设施-好",房间干净卫生；床品舒适；设备齐全,干净卫生；舒适；齐全,房间-舒适度-舒适
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,"房间-清洁程度-干净,房间-舒适度-舒适,房间-设施-好",房间干净卫生；床品舒适；设备齐全,干净卫生；舒适；齐全,房间-设施-好
1,2,前台接待的M美女很好,服务-态度-好,M美女很好,很好,服务-态度-好
2,3,位置好，房间现代、干净、漂亮！,"位置-便利性-交通-发达,房间-清洁程度-干净",位置好；房间干净,好；干净,位置-便利性-交通-发达
...,...,...,...,...,...,...
996,997,服务很好，还给孩子加了一张小床，给升级了房间，房间很大。客房服务很赞地理位置优越旁边吃的太多了,"服务-态度-好, 房间-大小-合适",服务很好，还给孩子加了一张小床；给升级了房间，房间很大,"服务好,加小床,升级房间",房间-大小-合适
997,998,性价比一般，服务还可以,"价格-合理性-一般, 服务-态度-好",性价比一般；服务还可以,"性价比一般,服务可以",价格-合理性-一般
997,998,性价比一般，服务还可以,"价格-合理性-一般, 服务-态度-好",性价比一般；服务还可以,"性价比一般,服务可以",服务-态度-好
998,999,酒店位置很不错，地铁就在楼下，自驾车到酒店，换乘地铁出行都很方便...,位置-便利性-交通-发达,酒店位置很不错，地铁就在楼下，自驾车到酒店，换乘地铁出行都很方便,"位置好,地铁近,出行方便",位置-便利性-交通-发达


In [92]:
# 创建空 DataFrame，方便之后计算
ans_df = pd.DataFrame()

# 对 1000 条评论，循环执行以下内容
for m in range(0, len(query_df)):

    # 如果评论内容非空，其实这个也没啥用
    if query_df['评论内容_new'].iloc[m] != '':
        
        ### huggingface
        #### 获取相似度分数

        # 获取第 m 个评论的向量
        q = query_df['query_embedding'].iloc[m]
        
        #### 在 FAQ 数据中循环计算相似度
        # df 是 FAQ 数据，所以这里获取 FAQ 向量并计算与评论向量的余弦相似度
        df["similarities"] = df['faq_embedding'].apply(lambda x: cosine_similarty(x[0], q[0]))

        # 对相似度进行排序
        cnt_df = df.sort_values("similarities", ascending=False)

        ##### 设置前 5 个相似的计数
        cntcnt_df = cnt_df.head(5)
        min_score = cntcnt_df['similarities'].iloc[4]  # 获取前 5 个相似中最小的相似度
        max_score = cntcnt_df['similarities'].iloc[0]  # 获取前 5 个相似中最大的相似度

        # 对分类结果进行分组并计数
        group_df = cntcnt_df.groupby('分类结果')['faq_new'].count().reset_index(name='cnt')
        group_df = group_df.sort_values("cnt", ascending=False)
        res = group_df.head(1)['分类结果'].iloc[0]  # 获取出现次数最多的分类结果
        ratio_score = group_df.head(1)['cnt'].iloc[0] / 5  # 计算该分类结果占前 5 个相似 FAQ 的比例

        # 将结果存入 ans_df
        ans_df.at[m, 'review_no'] = query_df['评论序号'].iloc[m]
        ans_df.at[m, 'query_long'] = query_df['评论内容'].iloc[m]
        ans_df.at[m, 'ngram'] = 1
        ans_df.at[m, 'query'] = query_df['评论内容_new'].iloc[m]
        ans_df.at[m, 'predict'] = res    
        ans_df.at[m, 'minscore'] = min_score  
        ans_df.at[m, 'maxscore'] = max_score          
        ans_df.at[m, 'ratio'] = ratio_score

        
ans_df = ans_df.rename(columns = {'review_no':'no'})


In [93]:
ans_df

,no,query_long,ngram,query,predict,minscore,maxscore,ratio
0,1.0,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,1.0,酒店挺不错的,服务-态度-好,0.885521,0.910004,0.8
1,1.0,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,1.0,房间干净卫生,房间-清洁程度-干净,0.899958,0.900690,1.0
2,1.0,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,1.0,设备齐全,房间-设施-好,0.844927,0.870343,1.0
3,1.0,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,1.0,床品舒适,房间-舒适度-舒适,0.898947,0.919234,1.0
4,1.0,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,1.0,早餐精致好吃,服务-态度-好,0.836675,0.858469,1.0
...,...,...,...,...,...,...,...,...
4458,999.0,酒店位置很不错，地铁就在楼下，自驾车到酒店，换乘地铁出行都很方便...,1.0,酒店位置很不错,位置-便利性-交通-发达,0.899407,0.916327,0.8
4459,999.0,酒店位置很不错，地铁就在楼下，自驾车到酒店，换乘地铁出行都很方便...,1.0,地铁就在楼下,位置-便利性-交通-发达,0.798527,0.826187,0.6
4460,999.0,酒店位置很不错，地铁就在楼下，自驾车到酒店，换乘地铁出行都很方便...,1.0,自驾车到酒店,位置-便利性-交通-发达,0.816214,0.821928,1.0
4461,999.0,酒店位置很不错，地铁就在楼下，自驾车到酒店，换乘地铁出行都很方便...,1.0,换乘地铁出行都很方便...,位置-便利性-交通-发达,0.814009,0.833386,0.8


In [94]:
############ parameters <---------------------
############ set the range here 
min_score_list = range_list(0.76, 0.8, 0.01)
max_score_list = range_list(0.82, 0.92, 0.01)
ratio_list = range_list(0.6, 0.9, 0.1)

############ measure - choice can be error_per_review or condprob_1_1 or condprob_0_0 or overall_accuracy
choice = 'error_per_review'

In [95]:
min_score_list
# max_score_list
# ratio_list

[0.76, 0.77, 0.78, 0.79, 0.8]

In [96]:
scheme_df['level1'] = scheme_df['level1'].ffill()
scheme_df['level2'] = scheme_df.groupby('level1')['level2'].ffill()
scheme_df['level3'] = scheme_df.groupby(['level1','level2'])['level3'].ffill()
scheme_df['level4'] = scheme_df.groupby(['level1','level2','level3'])['level4'].ffill()

scheme_df['cat'] = scheme_df.apply(scheme_level,axis=1)
col_names = scheme_df.cat.unique().tolist()
list_ = original['no'].dropna().unique().tolist()

In [97]:
scheme_df

,level1,level2,level3,level4,cat
0,房间,大小,偏大,NaN,房间-大小-偏大
1,房间,大小,合适,NaN,房间-大小-合适
2,房间,大小,偏小,NaN,房间-大小-偏小
3,房间,布局,合理,NaN,房间-布局-合理
4,房间,布局,不合理,NaN,房间-布局-不合理
5,房间,清洁程度,干净,NaN,房间-清洁程度-干净
6,房间,清洁程度,不干净,NaN,房间-清洁程度-不干净
7,房间,舒适度,舒适,NaN,房间-舒适度-舒适
8,房间,舒适度,不适,NaN,房间-舒适度-不适
9,房间,隔音水平,安静,NaN,房间-隔音水平-安静


In [98]:
def create_dataset(df, col, fact):
    
    dataset = pd.DataFrame(columns = col_names)
    
    for n in list_:
        tmp_df = df[df['no']==n]
        if len(tmp_df) > 0:
            res_list = tmp_df[col].dropna().unique().tolist()
            for each in col_names:
                if each in res_list:
                    dataset.at[n,each] = 1
                else:
                    dataset.at[n,each] = 0

        dataset.at[n,'no'] = n
        dataset.at[n,'fact'] = fact
    
    dataset = dataset.fillna(0)
    
    return (dataset)
    

In [99]:
def range_list(start, end, step):
    res = []
    current_value = start

    while current_value <= end:
        res.append(current_value)
        current_value  = current_value + step
        
    return res

In [100]:
def cal_error(col_names, true_result, pred_result):
    df = pd.DataFrame()
    for n in range(0, len(true_result)):
        cnt = 0
        for col in col_names:
            
            if true_result[col].iloc[n] != pred_result[col].iloc[n]:
                cnt = cnt + 1
                
        df.at[n, 'no'] = true_result['no'].iloc[n]
        df.at[n, 'error'] = cnt
        df.at[n, 'correct'] = len(col_names) - cnt
        df.at[n, 'total'] = len(col_names)
    return (df)

In [101]:
def cal_condprob(col_names, true_result, pred_result, given_val, find_val):
    df = pd.DataFrame()
    for n in range(0, len(true_result)):
        
        no = true_result['no'].iloc[n]
        cnt_pred = 0
        cnt_and = 0
        for col in col_names:
            if pred_result[col].iloc[n] == given_val:
                cnt_pred = cnt_pred + 1
                
            if true_result[col].iloc[n] == find_val and pred_result[col].iloc[n] == given_val:
                cnt_and = cnt_and + 1
        
        if cnt_pred == 0:
            prob = 0
        else:
            prob = cnt_and/cnt_pred

        df.at[n, 'no'] = no
        df.at[n, 'given_val'] = given_val
        df.at[n, 'find_val'] = find_val
        df.at[n, 'total_given_pred'] = cnt_pred
        df.at[n, 'total_satisfy'] = cnt_and
        #df.at[n, 'prob'] = prob
    return (df)

In [102]:
def special_lvl4(df):
    df = df.reset_index()
    for m in range(0, len(df)):
        split_list = df['predict'].iloc[m].split('-')
        if len(split_list) == 2:
            df.at[m,'level1'] = split_list[0]
            df.at[m,'level2'] = split_list[1]
        elif len(split_list) == 3:
            df.at[m,'level1'] = split_list[0]
            df.at[m,'level2'] = split_list[1]            
            df.at[m,'level3'] = split_list[2] 
        elif len(split_list) == 4:
            df.at[m,'level1'] = split_list[0]
            df.at[m,'level2'] = split_list[1]            
            df.at[m,'level3'] = split_list[2] 
            df.at[m,'level4'] = split_list[3] 
    return (df)
            

In [103]:
def match_result(row, choice):
    
    
    if type(row['predict_result']) != list:
        row['predict_result'] = []
    if choice == 1:
        ###both true and pred right
        res = list(set(row['true_result']) & set(row['predict_result']))
        
    elif choice == 2:
        ###only in true, but not in pred
        res = list(set(row['true_result']) - set(row['predict_result']))
        
    elif choice == 3:
        ###only in pred, but not in true
        res = list(set(row['predict_result']) - set(row['true_result']))
    
    return res 
        

In [104]:

def measure_sort(df, choice):
    if choice == 'error_per_review':
        df = df.sort_values(by=['error_per_review','condprob_1_1','overall_accuracy'], ascending = [True, False, False])
    elif choice == 'condprob_1_1':
        df = df.sort_values(by=['condprob_1_1','error_per_review','overall_accuracy'], ascending = [False, True, False])
    elif choice == 'overall_accuracy':
        df = df.sort_values(by=['overall_accuracy','error_per_review','condprob_1_1'], ascending = [False, True, False])
    elif choice == 'condprob_0_0':
        df = df.sort_values(by=['condprob_0_0','condprob_1_1','error_per_review','overall_accuracy'], ascending = [False, False, True, False])      
    return df 

In [105]:
original

,no,评论内容,分类结果,分类关键句,分类关键词,分类结果_new
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,"房间-清洁程度-干净,房间-舒适度-舒适,房间-设施-好",房间干净卫生；床品舒适；设备齐全,干净卫生；舒适；齐全,房间-清洁程度-干净
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,"房间-清洁程度-干净,房间-舒适度-舒适,房间-设施-好",房间干净卫生；床品舒适；设备齐全,干净卫生；舒适；齐全,房间-舒适度-舒适
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,"房间-清洁程度-干净,房间-舒适度-舒适,房间-设施-好",房间干净卫生；床品舒适；设备齐全,干净卫生；舒适；齐全,房间-设施-好
1,2,前台接待的M美女很好,服务-态度-好,M美女很好,很好,服务-态度-好
2,3,位置好，房间现代、干净、漂亮！,"位置-便利性-交通-发达,房间-清洁程度-干净",位置好；房间干净,好；干净,位置-便利性-交通-发达
...,...,...,...,...,...,...
996,997,服务很好，还给孩子加了一张小床，给升级了房间，房间很大。客房服务很赞地理位置优越旁边吃的太多了,"服务-态度-好, 房间-大小-合适",服务很好，还给孩子加了一张小床；给升级了房间，房间很大,"服务好,加小床,升级房间",房间-大小-合适
997,998,性价比一般，服务还可以,"价格-合理性-一般, 服务-态度-好",性价比一般；服务还可以,"性价比一般,服务可以",价格-合理性-一般
997,998,性价比一般，服务还可以,"价格-合理性-一般, 服务-态度-好",性价比一般；服务还可以,"性价比一般,服务可以",服务-态度-好
998,999,酒店位置很不错，地铁就在楼下，自驾车到酒店，换乘地铁出行都很方便...,位置-便利性-交通-发达,酒店位置很不错，地铁就在楼下，自驾车到酒店，换乘地铁出行都很方便,"位置好,地铁近,出行方便",位置-便利性-交通-发达


In [106]:
report_df = pd.DataFrame()
k=0
for min_ in min_score_list:
    for max_ in max_score_list:
        for ratio in ratio_list:
            

            ####to get distinct pred for each no and as well clear level4 (no dup lv3 with 4)
            filtered_df = ans_df[(ans_df['minscore']>=min_)&(ans_df['maxscore']>=max_)&(ans_df['ratio']>=ratio)]
            filtered_df = special_lvl4(filtered_df)
            filtered_df = filtered_df.sort_values(by=['no','query_long','predict','ratio','maxscore','minscore'], ascending = [True, False, False, False, False,False])
            filtered_df['rank'] = filtered_df.groupby(['no','query_long','predict']).cumcount(ascending=True) + 1
            filtered_df = filtered_df[filtered_df['rank']==1]
            filtered_df = filtered_df.sort_values(by=['no','ratio','maxscore','minscore'], ascending = [True, False, False, False])        
            filtered_df['rank2'] = filtered_df.groupby(['no','level1', 'level2', 'level3']).cumcount(ascending=True) + 1
            
            
            ####start comparing
            true_result = create_dataset(original, '分类结果_new', 'True_result')
            pred_result = create_dataset(filtered_df, 'predict', 'Predicted_result')

            result_df = cal_error(col_names, true_result, pred_result)
            result_df2_11 = cal_condprob(col_names, true_result, pred_result, 1,1)
            result_df2_00 = cal_condprob(col_names, true_result, pred_result, 0,0)
            
            
            report_df.at[k, 'min_score'] = min_
            report_df.at[k, 'max_score'] = max_
            report_df.at[k, 'ratio'] = ratio
            
            report_df.at[k, 'overall_accuracy'] = (result_df['correct'].sum())/(len(true_result) * len(col_names))
            report_df.at[k, 'error_per_review'] = (result_df['error'].sum())/(len(true_result))
            

            report_df.at[k, 'condprob_1_1'] = result_df2_11['total_satisfy'].sum()/result_df2_11['total_given_pred'].sum()
            report_df.at[k, 'condprob_0_0'] = (result_df2_00['total_satisfy'].sum()/result_df2_00['total_given_pred'].sum())
            
            k=k+1

/var/folders/0h/yp5_0sv92pb6b5x_2748bs380000gn/T/ipykernel_47456/4265081813.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset = dataset.fillna(0)
/var/folders/0h/yp5_0sv92pb6b5x_2748bs380000gn/T/ipykernel_47456/4265081813.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset = dataset.fillna(0)
/var/folders/0h/yp5_0sv92pb6b5x_2748bs380000gn/T/ipykernel_47456/4265081813.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) 

In [107]:
report_df_final = measure_sort(report_df, choice = choice )
report_df_final.to_excel('allsets.xlsx')

In [108]:
report_df_final.head(10)
        

,min_score,max_score,ratio,overall_accuracy,error_per_review,condprob_1_1,condprob_0_0
25,0.76,0.88,0.7,0.980659,0.851,0.608964,0.986206
26,0.76,0.88,0.8,0.980659,0.851,0.608964,0.986206
69,0.77,0.88,0.7,0.980659,0.851,0.608964,0.986206
70,0.77,0.88,0.8,0.980659,0.851,0.608964,0.986206
113,0.78,0.88,0.7,0.980636,0.852,0.608359,0.986184
114,0.78,0.88,0.8,0.980636,0.852,0.608359,0.986184
157,0.79,0.88,0.7,0.980614,0.853,0.608087,0.986138
158,0.79,0.88,0.8,0.980614,0.853,0.608087,0.986138
201,0.80,0.88,0.7,0.980591,0.854,0.607477,0.986116
202,0.80,0.88,0.8,0.980591,0.854,0.607477,0.986116


In [109]:
######find the best set (taje the firs)
best_min = report_df_final['min_score'].iloc[0]
best_max = report_df_final['max_score'].iloc[0]
best_ratio = report_df_final['ratio'].iloc[0]


print ('best_min:', best_min)
print ('best_max:', best_max)
print ('best_ratio:', best_ratio)

best_min: 0.76
best_max: 0.88
best_ratio: 0.7


In [110]:
#####display the reports for investigation
filtered_df = ans_df[(ans_df['minscore']>=best_min)&(ans_df['maxscore']>=best_max)&(ans_df['ratio']>=best_ratio)]
filtered_df = special_lvl4(filtered_df)
filtered_df = filtered_df.sort_values(by=['no','query_long','predict','ratio','maxscore','minscore'], ascending = [True, False, False, False, False,False])
filtered_df['rank'] = filtered_df.groupby(['no','query_long','predict']).cumcount(ascending=True) + 1
filtered_df = filtered_df[filtered_df['rank']==1]
filtered_df = filtered_df.sort_values(by=['no','ratio','maxscore','minscore'], ascending = [True, False, False, False])        
filtered_df['rank2'] = filtered_df.groupby(['no','level1', 'level2', 'level3']).cumcount(ascending=True) + 1
   
    
true_result = create_dataset(original, '分类结果_new', 'True_result')
pred_result = create_dataset(filtered_df, 'predict', 'Predicted_result')

result_df = cal_error(col_names, true_result, pred_result)
result_df = pd.merge(result_df, original.groupby(['no'])['分类结果_new'].apply(list).reset_index(name='true_result'), on ='no', how = 'left')
result_df = pd.merge(result_df, filtered_df.groupby(['no'])['predict'].apply(list).reset_index(name='predict_result'), on ='no', how = 'left')
result_df['correct_matches'] = result_df.apply(match_result, axis=1, choice = 1)   
result_df['true1_pred0'] = result_df.apply(match_result, axis=1, choice = 2)   
result_df['true0_pred1'] = result_df.apply(match_result, axis=1, choice = 3)   


result_df3 = original[['no','评论内容']].drop_duplicates()
result_df2 = filtered_df[['no', 'ngram', 'query']].drop_duplicates()
result_df2['query_n'] = result_df2['query'] + '_' + (result_df2['ngram']).astype(str)
result_df2 = result_df2.groupby(['no'])['query_n'].apply(list).reset_index(name='query_ngram')
result_df = pd.merge(result_df, result_df3, on = 'no', how = 'left')
result_df = pd.merge(result_df, result_df2, on = 'no', how = 'left')


result_df.to_excel('bestset.xlsx')




/var/folders/0h/yp5_0sv92pb6b5x_2748bs380000gn/T/ipykernel_47456/4265081813.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset = dataset.fillna(0)
/var/folders/0h/yp5_0sv92pb6b5x_2748bs380000gn/T/ipykernel_47456/4265081813.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset = dataset.fillna(0)


In [111]:
result_df


,no,error,correct,total,true_result,predict_result,correct_matches,true1_pred0,true0_pred1,评论内容,query_ngram
0,1.0,1.0,43.0,44.0,"[房间-清洁程度-干净, 房间-舒适度-舒适, 房间-设施-好]","[房间-舒适度-舒适, 房间-清洁程度-干净, 服务-态度-好]","[房间-舒适度-舒适, 房间-清洁程度-干净]",[房间-设施-好],[服务-态度-好],酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,"[床品舒适_1.0, 房间干净卫生_1.0, 酒店挺不错的_1.0]"
1,2.0,0.0,44.0,44.0,[服务-态度-好],[服务-态度-好],[服务-态度-好],[],[],前台接待的M美女很好,[前台接待的M美女很好_1.0]
2,3.0,2.0,42.0,44.0,"[位置-便利性-交通-发达, 房间-清洁程度-干净]",[房间-设施-好],[],"[位置-便利性-交通-发达, 房间-清洁程度-干净]",[房间-设施-好],位置好，房间现代、干净、漂亮！,[房间现代、干净、漂亮_1.0]
3,4.0,1.0,43.0,44.0,"[服务-态度-好, 房间-设施-好, 位置-便利性-交通-发达, 位置-便利性-附近设施多样...","[服务-态度-好, 位置-便利性-交通-发达]","[服务-态度-好, 位置-便利性-交通-发达]","[房间-设施-好, 位置-便利性-附近设施多样性-多]",[],"前台的态度很好，升级了行政房 房间设备非常人性化，卫生间是自动感应的,房间温度调试的很舒适，...","[前台的态度很好_1.0, 在会展中心的核心地带_1.0]"
4,5.0,0.0,44.0,44.0,[服务-态度-好],[服务-态度-好],[服务-态度-好],[],[],目前全国住过印象最好的酒店,[目前全国住过印象最好的酒店_1.0]
...,...,...,...,...,...,...,...,...,...,...,...
995,996.0,1.0,43.0,44.0,"[服务-态度-好, 房间-设施-好, 酒店额外设施-质量-好]",[服务-态度-好],[服务-态度-好],"[房间-设施-好, 酒店额外设施-质量-好]",[],饭店的服务、设施、餐点都非常满意,[饭店的服务、设施、餐点都非常满意_1.0]
996,997.0,2.0,42.0,44.0,"[服务-态度-好, 房间-大小-合适]","[服务-态度-好, 房间-大小-偏大]",[服务-态度-好],[房间-大小-合适],[房间-大小-偏大],服务很好，还给孩子加了一张小床，给升级了房间，房间很大。客房服务很赞地理位置优越旁边吃的太多了,"[服务很好_1.0, 房间很大_1.0]"
997,998.0,0.0,44.0,44.0,"[价格-合理性-一般, 服务-态度-好]",[服务-态度-好],[服务-态度-好],[价格-合理性-一般],[],性价比一般，服务还可以,[服务还可以_1.0]
998,999.0,0.0,44.0,44.0,[位置-便利性-交通-发达],[位置-便利性-交通-发达],[位置-便利性-交通-发达],[],[],酒店位置很不错，地铁就在楼下，自驾车到酒店，换乘地铁出行都很方便...,[酒店位置很不错_1.0]


In [112]:
#######using choice: condprob_1_1 
choice = 'condprob_1_1'
report_df_final = measure_sort(report_df, choice = choice )
report_df_final.head(10)

,min_score,max_score,ratio,overall_accuracy,error_per_review,condprob_1_1,condprob_0_0
43,0.76,0.92,0.9,0.978977,0.925,0.741007,0.979731
87,0.77,0.92,0.9,0.978977,0.925,0.741007,0.979731
131,0.78,0.92,0.9,0.978977,0.925,0.741007,0.979731
175,0.79,0.92,0.9,0.978977,0.925,0.741007,0.979731
219,0.80,0.92,0.9,0.978977,0.925,0.741007,0.979731
173,0.79,0.92,0.7,0.979477,0.903,0.703196,0.980859
174,0.79,0.92,0.8,0.979477,0.903,0.703196,0.980859
217,0.80,0.92,0.7,0.979477,0.903,0.703196,0.980859
218,0.80,0.92,0.8,0.979477,0.903,0.703196,0.980859
41,0.76,0.92,0.7,0.979455,0.904,0.700000,0.980859


In [113]:
#######using choice: condprob_1_1 
choice = 'overall_accuracy'
report_df_final = measure_sort(report_df, choice = choice )
report_df_final.head(10)

,min_score,max_score,ratio,overall_accuracy,error_per_review,condprob_1_1,condprob_0_0
25,0.76,0.88,0.7,0.980659,0.851,0.608964,0.986206
26,0.76,0.88,0.8,0.980659,0.851,0.608964,0.986206
69,0.77,0.88,0.7,0.980659,0.851,0.608964,0.986206
70,0.77,0.88,0.8,0.980659,0.851,0.608964,0.986206
113,0.78,0.88,0.7,0.980636,0.852,0.608359,0.986184
114,0.78,0.88,0.8,0.980636,0.852,0.608359,0.986184
157,0.79,0.88,0.7,0.980614,0.853,0.608087,0.986138
158,0.79,0.88,0.8,0.980614,0.853,0.608087,0.986138
201,0.80,0.88,0.7,0.980591,0.854,0.607477,0.986116
202,0.80,0.88,0.8,0.980591,0.854,0.607477,0.986116
